In [1]:
import gc
import os
import sys

import pandas as pd
import vaex
import numpy as np
from vaex.ml.sklearn import Predictor
import xgboost as xgb
from sklearn import model_selection as skl_ms
from sklearn.metrics import balanced_accuracy_score

main_dir = os.path.normpath(os.getcwd()+os.sep+os.pardir)
sys.path.append(main_dir)  # Add the main directory to sys.path
# from src.functions import data_prep as fu  # in order to import functions.py
# from src.functions import ml_prep as mp  # in order to import functions.py

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# os.environ['MAX_BATCH_SIZE'] = '32'
# import autokeras as ak

date_time_fmt = "%Y/%m/%d %H:%M:%S"

In [5]:
# Dataset prep
df_pd = pd.DataFrame(np.random.randint(2, size=(10**6, 21), dtype=np.uint8))  # vaex data
# df_pd = pd.read_csv("data/raw/creditcard.csv").drop(columns=["Time"]).pipe(fu.reduce_mem)  # real data for AK
gc.collect()

df_pd.columns = ["v_"+str(vname) for vname in list(df_pd.columns)]
target_vname = "Class"
df_pd.rename(columns={df_pd.columns[-1]: target_vname}, inplace=True, errors="ignore")

# vaex
features = df_pd.columns.drop([target_vname]).tolist()
df_vaex = vaex.from_pandas(df_pd, copy_index=False)
df_vaex_train, df_vaex_test = df_vaex.ml.train_test_split(test_size=0.3, verbose=False)

# pandas
df_pd.rename(columns={df_pd.columns[-1]: target_vname}, inplace=True, errors="ignore")
df_pd_train, df_pd_test = skl_ms.train_test_split(df_pd, test_size=0.3, random_state=123)
X_train, y_train = df_pd_train[features], df_pd_train[target_vname]
X_test, y_test = df_pd_test[features], df_pd_test[target_vname]

In [3]:
## XGB start
# start_time = pd.to_datetime("now")
# print("Start time:", start_time.strftime(date_time_fmt))
#
#
# # ML data prep
# X_train_xgb, X_test_xgb = mp.dmatricise(X_train, y_train), mp.dmatricise(X_test, y_test)
#
# params = {
#     "tree_method": "hist",
#     "objective": "binary:logistic",
#     "eval_metric": "logloss"
# }
# xgb_clf = xgb.XGBClassifier()
# xgb_param = xgb_clf.get_xgb_params()
# cvresult = xgb.cv(params, X_train_xgb, num_boost_round=100, nfold=10, stratified=True,
#        verbose_eval=False, early_stopping_rounds=5)
#
# xgb_evals_result = dict()
# xgb_clf = xgb.train(params, X_train_xgb, evals=[(X_train_xgb, "train"), (X_test_xgb, "test")],
#                     num_boost_round=cvresult.shape[0], verbose_eval=False, early_stopping_rounds=5,
#                     evals_result=xgb_evals_result)
# print("XGBoost logloss on test set:", xgb_clf.best_score)
##  XGBoost logloss on test set: 'logloss': 0.002031

## del xgb_clf, X_train_xgb, X_test_xgb
## gc.collect()

# finish_time = pd.to_datetime("now")
# print("Finish time:", finish_time.strftime(date_time_fmt), "or", (finish_time-start_time).seconds/60, "minutes.\n")

In [4]:
## Autokeras start
# date_time_fmt = "%Y/%m/%d %H:%M:%S"
# start_time = pd.to_datetime("now")
# print("Start time:", start_time.strftime(date_time_fmt))
#
# ak_clf = ak.StructuredDataClassifier(max_trials=5, loss='binary_crossentropy', objective="val_loss", seed=123)
# ak_clf.fit(X_train, y_train, epochs=50, verbose=0)
# print("AutoKeras logloss on test set:", ak_clf.evaluate(X_test, y_test))
# # AutoKeras logloss on test set: 0.0038057671467236565
#
# finish_time = pd.to_datetime("now")
# print("Finish time:", finish_time.strftime(date_time_fmt), "or", (finish_time-start_time).seconds/60, "minutes.\n")

In [17]:
## Vaex start
start_time = pd.to_datetime("now")
print("Start time:", start_time.strftime(date_time_fmt))

# ML
# print(type(ak_clf))
xgb_clf_vaex = xgb.XGBClassifier()
vaex_model = Predictor(model=xgb_clf_vaex, features=features, target=target_vname, prediction_name='pred')
vaex_model.fit(df=df_vaex_train)
vaex_preds = vaex_model.transform(df_vaex_test)
vaex_bacuracy = balanced_accuracy_score(y_test, vaex_preds.to_pandas_df()[target_vname], adjusted=True)
print("Balanced accuracy: %.2f%%" % (vaex_bacuracy * 100.0))

finish_time = pd.to_datetime("now")
print("Finish time:", finish_time.strftime(date_time_fmt), "or", round((finish_time-start_time).seconds/60, 2), "minutes.\n")

Start time: 2020/04/30 18:06:19
Balanced accuracy: -0.12%
Finish time: 2020/04/30 18:07:38 or 1.32 minutes.



In [18]:
## XGB start
start_time = pd.to_datetime("now")
print("Start time:", start_time.strftime(date_time_fmt))

# ML
xgb_clf_pd = xgb.XGBClassifier()
xgb_model = xgb_clf_pd.fit(X_train, y_train)
pd_preds = xgb_model.predict(X_test)
# pd_preds = [round(value) for value in pd_preds]
pd_bacuracy = balanced_accuracy_score(y_test, pd_preds, adjusted=True)
print("Balanced accuracy: %.2f%%" % (pd_bacuracy * 100.0))

finish_time = pd.to_datetime("now")
print("Finish time:", finish_time.strftime(date_time_fmt), "or", round((finish_time-start_time).seconds/60, 2), "minutes.\n")

Start time: 2020/04/30 18:07:38
Balanced accuracy: 0.04%
Finish time: 2020/04/30 18:09:02 or 1.4 minutes.

